# Convolutional Neural Network

### Importing the libraries

In [1]:
!pip install --upgrade tensorflow keras Pillow

Requirement already up-to-date: tensorflow in /home/vitorlamego/anaconda3/lib/python3.8/site-packages (2.6.0)
Requirement already up-to-date: keras in /home/vitorlamego/anaconda3/lib/python3.8/site-packages (2.6.0)
Requirement already up-to-date: Pillow in /home/vitorlamego/anaconda3/lib/python3.8/site-packages (8.3.1)


In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [23]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   width_shift_range=0.4,
                                   height_shift_range=0.4,
                                   horizontal_flip = True,
                                   vertical_flip=True,
                                   rotation_range=260)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64,64),
                                                 batch_size = 32,
                                                 class_mode = 'binary'
                                                )


Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [24]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [25]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [26]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, activation='elu', input_shape=[64, 64, 3]))

In [27]:
cnn.add(tf.keras.layers.Dropout(rate=0.25))

### Step 2 - Pooling

In [28]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

### Adding a second convolutional layer

In [29]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, activation='elu'))
cnn.add(tf.keras.layers.Dropout(rate=0.25))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

### Step 3 - Flattening

In [30]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [31]:
cnn.add(tf.keras.layers.Dense(units=128, activation='elu'))

### Step 5 - Output Layer

In [32]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [33]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [34]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 180s 713ms/step - loss: 1.0407 - accuracy: 0.5034 - val_loss: 1.1187 - val_accuracy: 0.5585
Epoch 2/25
250/250 [==============================] - 123s 493ms/step - loss: 0.6950 - accuracy: 0.4927 - val_loss: 1.1897 - val_accuracy: 0.5850
Epoch 3/25
250/250 [==============================] - 124s 497ms/step - loss: 0.6945 - accuracy: 0.4989 - val_loss: 0.9762 - val_accuracy: 0.5650
Epoch 4/25
250/250 [==============================] - 115s 458ms/step - loss: 0.6950 - accuracy: 0.4916 - val_loss: 0.7665 - val_accuracy: 0.6005
Epoch 5/25
250/250 [==============================] - 128s 511ms/step - loss: 0.6952 - accuracy: 0.4989 - val_loss: 0.6987 - val_accuracy: 0.5810
Epoch 6/25
250/250 [==============================] - 120s 481ms/step - loss: 0.6947 - accuracy: 0.4996 - val_loss: 0.7634 - val_accuracy: 0.5000
Epoch 7/25
250/250 [==============================] - 123s 490ms/step - loss: 0.6957 - accuracy: 0.5013 - val_loss: 0.6955 -

## Part 4 - Making a single prediction

In [37]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [38]:
print(prediction)

cat


## Fim